In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [2]:
dataset = datasets.ImageFolder(
    root='./datasets/emojis',       
)

# Create separate transformation pipelines for the training and the testing data (due to augmentation)
train_transform= transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((64, 64)),  
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomRotation(degrees=20), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform= transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Split the dataset into training and testing datasets in the ratio 4: 1
train_dataset, test_dataset= random_split(dataset, [432, 108])
train_dataset.dataset.transform= train_transform
test_dataset.dataset.transform= test_transform
train_data= DataLoader(train_dataset, batch_size=32, shuffle=True)
test_data= DataLoader(test_dataset, batch_size=32)

In [3]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1), # Convolutional Layer 1
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.1),
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1), # Convolutional Layer 2
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1), # Convolutional Layer 3
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),          
            nn.Flatten(),

            nn.Linear(64 * 8 * 8, 128), # Dense Layer 1
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(128, 18) # Dense Layer 2
        )

    def forward(self, x):
        return self.model(x) # Pass the input through the whole architecture

In [4]:
model= CNN()
cross_entropy= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model for 50 epochs 
for e in range(50):
    model.train()
    loss, correct, total= 0, 0, 0
    for images, labels in train_data:
        optimizer.zero_grad() # Clear the previous gradients
        outputs= model(images)
        current_loss = cross_entropy(outputs, labels)
        current_loss.backward() # Backpropagation
        optimizer.step()

        loss+= current_loss
        scores, indices = torch.max(outputs, dim= 1)
        total+= labels.size(0)
        correct+= (indices == labels).sum()
    
    # Print the results after every 10 epochs
    if (e%10==9 and e!=49):
        accuracy= 100*correct.item()/total
        print(f"Current Iteration: {e+1}, Current Loss: {loss.item():.2f}, Current Accuracy: {accuracy:.2f}%")

# Print the final results
train_accuracy= 100*correct.item()/total
print(f"Final Iteration: {e+1}, Final Loss: {loss.item():.2f}, Final Train Accuracy: {train_accuracy:.2f}%")

Current Iteration: 10, Current Loss: 40.26, Current Accuracy: 8.10%
Current Iteration: 20, Current Loss: 21.75, Current Accuracy: 47.45%
Current Iteration: 30, Current Loss: 15.90, Current Accuracy: 62.50%
Current Iteration: 40, Current Loss: 12.39, Current Accuracy: 67.82%
Final Iteration: 50, Final Loss: 10.80, Final Train Accuracy: 75.23%


In [6]:
model.eval()  
correct, total= 0, 0

# Disable gradient calculation for inference
with torch.no_grad(): 
    for images, labels in test_data:
        outputs = model(images)
        scores, indices = torch.max(outputs, dim= 1)
        total += labels.size(0)
        correct += (indices == labels).sum()

test_accuracy = 100*correct.item()/total
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 67.59%
